In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from scipy.stats import hmean
import pickle
from sklearn.utils import resample

In [ ]:
Train_Features = [
    "ticker", "company_name", "sector", "industry", "market_cap",
    "price", "open", "close", "high", "low",
    "volume", "adj_close", "dividend", "dividend_yield", "pe_ratio",
    "eps", "beta", "52_week_high", "52_week_low", "shares_outstanding",
    "float", "average_volume", "market", "exchange", "isin",
    "cusip", "country", "currency", "ipo_date", "earnings_date",
    "revenue", "cost_of_goods_sold", "gross_profit", "operating_expenses", "operating_income",
    "ebit", "ebitda", "net_income", "income_before_tax", "tax_expense",
    "net_income_applicable_to_common_shares", "basic_eps", "diluted_eps", "total_assets", "current_assets",
    "non_current_assets", "total_liabilities", "current_liabilities", "non_current_liabilities", "shareholders_equity",
    "retained_earnings", "cash_and_cash_equivalents", "short_term_investments", "long_term_investments", "inventory",
    "accounts_receivable", "accounts_payable", "depreciation", "amortization", "capital_expenditures",
    "loan_id", "loan_amount", "loan_term", "interest_rate", "installment",
    "issue_date", "loan_status", "payment_status", "borrower_score", "borrower_income",
    "debt_to_income", "employment_length", "purpose", "home_ownership", "delinquency_2yrs",
    "credit_score", "fico_range_low", "fico_range_high", "revol_util", "num_open_credit_lines",
    "total_credit_lines", "public_records", "collections_12_mths_ex_med", "application_type", "verification_status",
    "bond_id", "bond_name", "maturity_date", "coupon_rate", "yield_to_maturity",
    "face_value", "issue_price", "current_price", "duration", "convexity",
    "credit_rating", "issuer", "callable", "puttable", "bond_type",
    "transaction_id", "transaction_date", "transaction_amount", "transaction_type", "merchant_name",
    "merchant_category", "account_id", "balance_before", "balance_after", "location",
    "crypto_symbol", "crypto_name", "market_rank", "circulating_supply", "total_supply",
    "max_supply", "market_dominance", "all_time_high", "all_time_low", "last_updated",
    "block_time", "hashing_algorithm", "platform", "explorer_url", "trading_pairs",
    "exchange_rate", "currency_pair", "base_currency", "quote_currency", "rate_date",
    "rate_time", "daily_change", "monthly_change", "yearly_change", "volume_24h",
    "investment_id", "investment_type", "investment_amount", "investment_date", "current_value",
    "gain_loss", "annual_return", "investment_duration", "investment_strategy", "fund_manager",
    "fund_id", "fund_name", "nav", "expense_ratio", "inception_date",
    "fund_category", "assets_under_management", "benchmark_index", "turnover_ratio", "dividend_distribution",
    "gdp", "inflation_rate", "unemployment_rate", "federal_funds_rate", "consumer_price_index",
    "producer_price_index", "retail_sales", "housing_starts", "trade_balance", "government_debt",
    "current_account_balance", "budget_deficit", "foreign_reserves", "money_supply", "taxpayer_id",
    "income_bracket", "taxable_income", "effective_tax_rate", "tax_paid", "deductions",
    "credits", "filing_status", "tax_year", "refund_amount", "bank_id",
    "branch_id", "account_type", "account_open_date", "account_balance", "interest_earned",
    "overdraft_limit", "minimum_balance", "monthly_fee", "account_status", "user_id",
    "customer_id", "registration_date", "last_login", "kyc_status", "risk_score",
    "fraud_flag", "device_id", "ip_address", "login_location", "portfolio_id",
    "asset_class", "allocation_percentage", "benchmark_return", "tracking_error", "sharpe_ratio",
    "alpha", "beta_coefficient", "standard_deviation", "max_drawdown", "audit_status", "accounting_standard", "financial_statement_type", "reporting_currency", "adjustment_reason",
    "deferred_tax_assets", "deferred_tax_liabilities", "intangible_assets", "goodwill", "preferred_equity",
    "policy_id", "policy_holder", "premium_amount", "coverage_amount", "claim_id",
    "claim_status", "underwriting_score", "risk_class", "loss_ratio", "combined_ratio",
    "order_id", "trade_price", "trade_volume", "order_type", "execution_time",
    "bid_price", "ask_price", "spread", "order_book_depth", "trading_halt",
    "swift_code", "iban", "routing_number", "account_opening_method", "branch_location",
    "atm_withdrawals", "wire_transfers", "monthly_statements", "account_tier", "fee_structure",
    "credit_limit", "credit_line_type", "charge_off_status", "days_past_due", "collection_agency",
    "restructuring_status", "forbearance_flag", "loan_purpose", "collateral_type", "repayment_behavior",
    "employment_rate", "labor_force_participation", "consumer_confidence_index", "housing_index", "manufacturing_index",
    "import_volume", "export_volume", "interest_payment", "sovereign_rating", "external_debt",
    "payment_method", "payment_gateway", "settlement_status", "refund_status", "dispute_id",
    "chargeback_amount", "recurring_payment", "subscription_id", "billing_cycle", "invoice_date",
    "wealth_segment", "advisor_id", "fee_schedule", "client_risk_profile", "discretionary_mandate",
    "goals_based_plan", "financial_goal", "investment_objective", "cash_allocation", "equity_allocation",
    "esg_score", "carbon_emission", "sustainability_rating", "board_diversity", "executive_compensation_ratio"
]

In [ ]:
Test_Features = [
   "revenue_growth", "net_margin", "operating_margin", "book_value_per_share", "enterprise_value",
   "ev_to_ebitda", "price_to_free_cash_flow", "fcf_margin", "roic", "roa",
   "cash_conversion_cycle", "interest_coverage_ratio", "days_payable_outstanding", "inventory_turnover", "quick_ratio",
   "z_score", "altman_z_score", "short_interest_ratio", "put_call_ratio", "analyst_recommendation",
   "price_target_high", "price_target_low", "estimate_revision", "guidance_change", "buyback_yield",
   "s&p_rating", "moody_rating", "recovery_rate", "default_probability", "credit_spread",
   "real_interest_rate", "velocity_of_money", "consumer_sentiment_index", "labor_cost_index", "construction_spending",
   "crypto_funding_rate", "staking_yield", "token_burn_rate", "dao_votes", "mining_difficulty",
   "digital_wallet_id", "transaction_fee", "payment_token", "subscription_status", "auto_renew_flag",
   "claim_frequency", "premium_to_coverage_ratio", "lapse_rate", "policy_duration", "actuarial_value"
]


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Define preprocessing function
def preprocess_keyword(keyword):
    return keyword.replace("_", " ").lower()

# Preprocess training and testing features
processed_keywords = [preprocess_keyword(k) for k in Train_Features]
processed_testing_keywords = [preprocess_keyword(k) for k in Test_Features]

# Load FinancialBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModel.from_pretrained("ProsusAI/finbert").to(device)

# Function to generate embeddings using FinancialBERT
def generate_embeddings(keywords, tokenizer, model, device):
    inputs = tokenizer(keywords, padding=True, truncation=True, return_tensors="pt", max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the [CLS] token embedding as the sentence representation
    cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return cls_embeddings

# Generate embeddings for training and testing data
embeddings = generate_embeddings(processed_keywords, tokenizer, model, device)
testing_embeddings = generate_embeddings(processed_testing_keywords, tokenizer, model, device)

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(embeddings)

# Perform t-SNE visualization
tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=42)
reduced_embeddings = tsne.fit_transform(embeddings)

# Plot t-SNE visualization
plt.figure(figsize=(16, 12))
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], alpha=0.7)

# Annotate each point with its corresponding label
for i, label in enumerate(processed_keywords):
    plt.annotate(label, (reduced_embeddings[i, 0], reduced_embeddings[i, 1]), fontsize=7, alpha=0.8)

# Add title and grid
plt.title("t-SNE Visualization of Financial Keyword Embeddings using FinancialBERT", fontsize=14)
plt.grid(True)
plt.show()


In [ ]:
# Initialize the StandardScaler
scaler = StandardScaler()
# Initialize the SVM classifier
svm_model = SVC(kernel='rbf', random_state=42, gamma='scale')

threshold_candidates = np.arange(0.8, 0.9, 0.01)

best_threshold = None
best_f1 = 0

min_positives_ratio = 0.01
max_positives_ratio = 0.10


for threshold in threshold_candidates:
    X_temp, y_temp = [], []

    for i in range(len(embeddings)):
        for j in range(i + 1, len(embeddings)):
            vec1, vec2 = embeddings[i], embeddings[j]
            cos_sim = cosine_similarity([vec1], [vec2])[0][0]
            label = 1 if cos_sim > threshold else 0
            X_temp.append(np.concatenate([vec1, vec2]))
            y_temp.append(label)

    X_temp, y_temp = np.array(X_temp), np.array(y_temp)
    positives_ratio = np.mean(y_temp)

    # Skip thresholds with too few or too many positives
    if not (min_positives_ratio <= positives_ratio <= max_positives_ratio):
        print(f"⚠️ Skipping threshold {threshold:.3f} due to positive ratio: {positives_ratio:.4f}")
        continue

    # Split and scale
    X_train_sub, X_val_sub, y_train_sub, y_val_sub = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)
    X_train_scaled = scaler.fit_transform(X_train_sub)
    X_val_scaled = scaler.transform(X_val_sub)

    # Train and evaluate
    svm_model.fit(X_train_scaled, y_train_sub)
    y_pred_val = svm_model.predict(X_val_scaled)
    f1 = f1_score(y_val_sub, y_pred_val, zero_division=0)

    print(f"✅ Threshold: {threshold:.3f} | F1: {f1:.4f} | Positive Ratio: {positives_ratio:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

if best_threshold is not None:
    print(f"\n🎯 Best Threshold Found: {best_threshold:.3f} with F1-score: {best_f1:.4f}")
else:
    print("\n❌ No valid threshold found within ratio constraints. Try adjusting the limits.")


In [ ]:
# Threshold for compatibility
threshold = 0.85

# Generate all possible pairs of features
X = []  # Input features (concatenated embeddings)
y = []  # Labels (1 for compatible, 0 for incompatible)

for i in range(len(embeddings)):
    for j in range(i + 1, len(embeddings)):
        combined_features = np.concatenate([embeddings[i], embeddings[j]])  # Concatenate embeddings
        X.append(combined_features)
        
        # Compute cosine similarity and assign label
        cos_sim = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
        y.append(1 if cos_sim > threshold else 0)

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

print(f"Original pairs: {len(X)}")

In [52]:
def jitter_embedding(embedding, noise_level=0.05):
    noise = np.random.normal(0, noise_level, embedding.shape)
    return embedding + noise

augmented_pairs = []
augmented_labels = []

n_augments = 3  # Number of jittered copies per compatible pair

for i in range(len(embeddings)):
    for j in range(i + 1, len(embeddings)):
        sim = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
        if sim > threshold:
            for _ in range(n_augments):
                vec1_jit = jitter_embedding(embeddings[i])
                vec2_jit = jitter_embedding(embeddings[j])
                augmented_pairs.append(np.concatenate([vec1_jit, vec2_jit]))
                augmented_pairs.append(np.concatenate([embeddings[i], vec2_jit]))
                augmented_pairs.append(np.concatenate([vec1_jit, embeddings[j]]))
                augmented_labels.extend([1, 1, 1])



print(f"Augmented pairs: {len(augmented_pairs)}")
print(f"Original pairs: {len(X)}")
# Combine original and augmented data
X_combined = np.vstack([X, np.array(augmented_pairs)])
y_combined = np.concatenate([y, np.array(augmented_labels)])

Augmented pairs: 29169
Original pairs: 43071


In [54]:
# Threshold for compatibility
threshold = 0.85

# Generate all possible pairs of features
X_Test = []  # Input features (concatenated embeddings)
y_test = []  # Labels (1 for compatible, 0 for incompatible)

for i in range(len(testing_embeddings)):
    for j in range(i + 1, len(testing_embeddings)):
        combined_features = np.concatenate([testing_embeddings[i], testing_embeddings[j]])  # Concatenate embeddings
        X_Test.append(combined_features)
        
        # Compute cosine similarity and assign label
        cos_sim = cosine_similarity([testing_embeddings[i]], [testing_embeddings[j]])[0][0]
        y_test.append(1 if cos_sim > threshold else 0)

# Convert to NumPy arrays
X_Test = np.array(X_Test)
y_test = np.array(y_test)

In [55]:
augmented_pairs = []
augmented_labels = []

n_augments = 3  # Number of jittered copies per compatible pair

for i in range(len(testing_embeddings)):
    for j in range(i + 1, len(testing_embeddings)):
        sim = cosine_similarity([testing_embeddings[i]], [testing_embeddings[j]])[0][0]
        if sim > threshold:
            for _ in range(n_augments):
                vec1_jit = jitter_embedding(testing_embeddings[i])
                vec2_jit = jitter_embedding(testing_embeddings[j])
                augmented_pairs.append(np.concatenate([vec1_jit, vec2_jit]))
                augmented_pairs.append(np.concatenate([testing_embeddings[i], vec2_jit]))
                augmented_pairs.append(np.concatenate([vec1_jit, testing_embeddings[j]]))
                augmented_labels.extend([1, 1, 1])


print(f"Augmented pairs: {len(augmented_pairs)}")
print(f"Original pairs: {len(X_Test)}")
# Combine original and augmented data
X_Test_combined = np.vstack([X_Test, np.array(augmented_pairs)])
y_Test_combined = np.concatenate([y_test, np.array(augmented_labels)])

Augmented pairs: 621
Original pairs: 1225


In [56]:
scaler = StandardScaler()
# Fit the scaler on the training data and transform both training and testing data
X_train_scaled = scaler.fit_transform(X_combined)
X_test_scaled = scaler.transform(X_Test_combined)

In [ ]:
# Define parameter grid for multiple kernels
param_grid = [
    {
        'kernel': ['linear'],
        'C': [0.1, 1, 10]
    },
    {
        'kernel': ['rbf'],
        'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto', 0.1, 1]
    },
    {
        'kernel': ['poly'],
        'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto', 0.1, 1],
        'degree': [2, 3, 4]
    }
]

# Initialize the Grid Search
svm = SVC(random_state=42)

grid_search = GridSearchCV(
    estimator=svm,
    param_grid=param_grid,
    cv=2,
    scoring='f1',
    verbose=2,
    n_jobs=-1
)

# Run the grid search
grid_search.fit(X_train_scaled, y)

results = grid_search.cv_results_
for mean, std, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"✔️ Params: {params} | Recall: {mean:.4f} (+/- {std:.4f})")

# Output best model
print(f"\n✅ Best Parameters: {grid_search.best_params_}")
best_model = grid_search.best_estimator_


In [58]:
# Define parameter grid for multiple kernels
param_grid = [
    {
        'kernel': ['rbf'],
        'C': list(range(1, 16, 1)),
        'gamma': ['scale', 'auto']
    },
    {
        'kernel': ['poly'],
        'C': list(range(1, 16, 1)),
        'gamma': ['scale', 'auto'],
        'degree': [2, 3]
    }
]

# Initialize the Grid Search
svm = SVC(random_state=42)

grid_search = GridSearchCV(
    estimator=svm,
    param_grid=param_grid,
    cv=2,
    scoring='recall',
    verbose=2,
    n_jobs=-1
)

# Run the grid search
grid_search.fit(X_train_scaled, y_combined)

results = grid_search.cv_results_
for mean, std, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"✔️ Params: {params} | Recall: {mean:.4f} (+/- {std:.4f})")

# Output best model
print(f"\n✅ Best Parameters: {grid_search.best_params_}")
best_model = grid_search.best_estimator_


Fitting 2 folds for each of 90 candidates, totalling 180 fits


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[CV] END ........................C=2, gamma=auto, kernel=rbf; total time=59.2min
[CV] END .......................C=2, gamma=scale, kernel=rbf; total time=60.0min
[CV] END ........................C=6, gamma=auto, kernel=rbf; total time=60.3min
[CV] END ........................C=1, gamma=auto, kernel=rbf; total time=60.4min
[CV] END .......................C=5, gamma=scale, kernel=rbf; total time=60.6min
[CV] END .......................C=1, gamma=scale, kernel=rbf; total time=60.7min
[CV] END .......................C=6, gamma=scale, kernel=rbf; total time=60.8min
[CV] END .......................C=4, gamma=scale, kernel=rbf; total time=60.9min
[CV] END ........................C=4, gamma=auto, kernel=rbf; total time=61.1min
[CV] END .......................C=2, gamma=scale, kernel=rbf; total time=71.5min
[CV] END .......................C=3, gamma=scale, kernel=rbf; total time=71.6min
[CV] END ........................C=3, gamma=auto, kernel=rbf; total time=71.7min
[CV] END ...................

In [59]:
# model = SVC(kernel='rbf', C=1, gamma='scale', random_state=42)
# model.fit(X_train_scaled, y_combined)

# Predict using best model
y_pred = best_model.predict(X_test_scaled)

# Evaluation
print("\n📊 Classification Report (Test Set):")
print(classification_report(y_Test_combined, y_pred, zero_division=0))


📊 Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      1156
           1       0.88      0.69      0.78       690

    accuracy                           0.85      1846
   macro avg       0.86      0.82      0.83      1846
weighted avg       0.85      0.85      0.85      1846



In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
def extract_columns(csv_path):
    raw_columns = list(pd.read_csv(csv_path, nrows=0).columns)
    return [preprocess_keyword(col) for col in raw_columns]

# Step 1: Load and preprocess columns
columns_1 = extract_columns("Datasets/Banking/dataset1/loan_data.csv")       # Dataset A
columns_2 = extract_columns("Datasets/Loans/dataset2/loan_data.csv")  # Dataset B (note: different folder!)

# Step 2: Generate embeddings (assuming your function works correctly)
embeddings_1 = generate_embeddings(columns_1, tokenizer, model, device)
embeddings_2 = generate_embeddings(columns_2, tokenizer, model, device)

# Step 3: Create pair embeddings between each column from dataset 1 and dataset 2
Dataset_test = []
column_pairs = []

for i, emb1 in enumerate(embeddings_1):
    for j, emb2 in enumerate(embeddings_2):
        pair_embedding = np.concatenate([emb1, emb2])
        Dataset_test.append(pair_embedding)
        column_pairs.append((columns_1[i], columns_2[j]))

Dataset_test = np.array(Dataset_test)

# Step 4: Predict with trained model
Dataset_test_scaled = scaler.transform(Dataset_test)
predictions = svm_model.predict(Dataset_test_scaled)

# Step 5: Extract compatible pairs
compatible_pairs = [(col1, col2) for (col1, col2), pred in zip(column_pairs, predictions) if pred == 1]

# Step 6: Compatibility score based on bidirectional coverage
matched_A = set(col1 for col1, _ in compatible_pairs)
matched_B = set(col2 for _, col2 in compatible_pairs)

coverage_A = len(matched_A) / len(columns_1)
coverage_B = len(matched_B) / len(columns_2)

# Use harmonic mean for stricter measure (penalizes one-sided matching)
if coverage_A > 0 and coverage_B > 0:
    compatibility_score = hmean([coverage_A, coverage_B])
else:
    compatibility_score = 0.0

# Step 7: Print results
print(f"\nCompatibility Score: {compatibility_score:.2f}")
print("Datasets are COMPATIBLE" if compatibility_score >= 0.7 else "Datasets are NOT compatible")

print("\n🔗 Compatible Column Pairs:")
if compatible_pairs:
    for col1, col2 in compatible_pairs:
        print(f"- {col1}  ↔️  {col2}")
else:
    print("No compatible columns found.")
